# Review data preparation

In [1]:
%load_ext autoreload
%autoreload 2

# Import ds libraries
import pandas as pd
import numpy as np
import re

# Import acquire functions
import nick_acquire as a
import nick_prepare as prep

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nickolaspedrimiranda/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/nickolaspedrimiranda/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [33]:
api_reviews = pd.read_csv('reviews_progress.csv')
scrape_reviews = pd.read_csv('scrapted_reviews.csv')

In [27]:
scrape_reviews.head(3)

,id,id_review,caption,relative_date,retrieval_date,rating,username,n_review_user,url_user,r_additional
0,41375676,ChZDSUhNMG9nS0VJQ0FnSUNGMDhHSkVnEAE,NaN,11 hours ago,2023-11-02 02:44:04.226505,5.0,Eddie Trujillo,1,https://www.google.com/maps/contrib/1064176602...,"['Service: 5', 'Atmosphere: 5']"
1,41375676,ChdDSUhNMG9nS0VJQ0FnSUNGM2NhcGpnRRAB,NaN,a day ago,2023-11-02 02:44:04.227451,5.0,Mahamadou Asouman,0,https://www.google.com/maps/contrib/1078602531...,"['Service: 5', 'Atmosphere: 5']"
2,41375676,ChdDSUhNMG9nS0VJQ0FnSUNGbmVidnRRRRAB,NaN,a day ago,2023-11-02 02:44:04.227954,5.0,sedjnie ritoe,0,https://www.google.com/maps/contrib/1170344005...,[]


In [34]:
scrape_reviews.relative_date = scrape_reviews.relative_date.apply(lambda x: x[:-4])
scrape_reviews.relative_date = ['1 years' if date == 'a year' else date for date in scrape_reviews.relative_date]
scrape_reviews.head(3)

,id,id_review,caption,relative_date,retrieval_date,rating,username,n_review_user,url_user,r_additional
0,41375676,ChZDSUhNMG9nS0VJQ0FnSUNGMDhHSkVnEAE,NaN,11 hours,2023-11-02 02:44:04.226505,5.0,Eddie Trujillo,1,https://www.google.com/maps/contrib/1064176602...,"['Service: 5', 'Atmosphere: 5']"
1,41375676,ChdDSUhNMG9nS0VJQ0FnSUNGM2NhcGpnRRAB,NaN,a day,2023-11-02 02:44:04.227451,5.0,Mahamadou Asouman,0,https://www.google.com/maps/contrib/1078602531...,"['Service: 5', 'Atmosphere: 5']"
2,41375676,ChdDSUhNMG9nS0VJQ0FnSUNGbmVidnRRRRAB,NaN,a day,2023-11-02 02:44:04.227954,5.0,sedjnie ritoe,0,https://www.google.com/maps/contrib/1170344005...,[]


In [10]:
scrape_reviews.id.unique()

array([41375676, 50093783, 50072407, 40401445, 50072544, 41585586,
       50105651, 50014375, 50110343, 41425365, 50035490, 50019166,
       50060865, 41681629, 50041581, 50005481, 50018654, 50085183,
       50033274, 50044965, 50127886])

In [80]:
p1 = scrape_reviews[scrape_reviews.id == 50093783]
review_counts = pd.DataFrame(p1.relative_date.value_counts())
review_counts['increment'] = 365/review_counts.relative_date
review_counts[:10]

,relative_date,increment
4 years,62,5.887097
3 years,34,10.735294
1 years,30,12.166667
2 years,20,18.250000
5 years,16,22.812500
10 months,4,91.250000
2 months,4,91.250000
11 months,3,121.666667
8 months,3,121.666667
6 months,3,121.666667


In [50]:
review_counts.loc['4 years'].increment

5.887096774193548

In [67]:
restaurant = scrape_reviews[scrape_reviews.id == 50093783].copy()

place = scrape_reviews[scrape_reviews.id == 50093783]
review_counts = pd.DataFrame(place.relative_date.value_counts())
review_counts['increment'] = 365/review_counts.relative_date

new_dates = []
i = 0
previous_year = '1 years'
for date in restaurant.relative_date:  # For every date
    if 'years' in date:
        if date != previous_year:
            i = 0
            previous_year = date
        adjusted_date = (365*(int(date[0]))) + (review_counts.loc[date].increment * i)
        i += 1
        new_dates.append(adjusted_date)
    else:
        new_dates.append(date)
restaurant['new_date'] = new_dates
restaurant

,id,id_review,caption,relative_date,retrieval_date,rating,username,n_review_user,url_user,r_additional,new_date
200,50093783,ChdDSUhNMG9nS0VJQ0FnSUNGcGY3VndRRRAB,The plantains were made on the spot in a rush ...,3 days,2023-11-02 02:47:30.895865,3.0,Six Seven Entertainment,0,https://www.google.com/maps/contrib/1068951192...,"['Service Take out', 'Price per person $10–20'...",3 days
201,50093783,ChdDSUhNMG9nS0VJQ0FnSUQtaWN1STBBRRAB,"It was my first time at this restaurant, and w...",2 weeks,2023-11-02 02:47:30.896429,1.0,S W,0,https://www.google.com/maps/contrib/1143606720...,[],2 weeks
202,50093783,ChZDSUhNMG9nS0VJQ0FnSUNaNWJpaUZBEAE,"I went there to get breakfast, at exactly 8am ...",a month,2023-11-02 02:47:30.897581,1.0,Delicia Singh,0,https://www.google.com/maps/contrib/1064364911...,"['Service Take out', 'Meal type Breakfast', 'P...",a month
203,50093783,ChZDSUhNMG9nS0VJQ0FnSUNac3JpM013EAE,Ordered online thru door dash and they sent th...,a month,2023-11-02 02:47:30.898268,1.0,Ashley White,0,https://www.google.com/maps/contrib/1160471953...,"['Food: 1', 'Service: 1', 'Atmosphere: 1']",a month
204,50093783,ChdDSUhNMG9nS0VJQ0FnSURwNTVhU3NBRRAB,NaN,2 months,2023-11-02 02:47:30.899326,3.0,Bellman Rodriguez Quiroga,63,https://www.google.com/maps/contrib/1147885710...,"['Meal type Lunch', 'Price per person $10–20',...",2 months
...,...,...,...,...,...,...,...,...,...,...,...
386,50093783,ChZDSUhNMG9nS0VJQ0FnSUNBejUzd0pnEAE,I bought three chickens and the chickens were ...,5 years,2023-11-02 02:49:53.843048,1.0,Mr.García,6,https://www.google.com/maps/contrib/1065367642...,[],2075.9375
387,50093783,ChdDSUhNMG9nS0VJQ0FnSUN3b3RUdTdRRRAB,From what I'm told the food supposed to be pr...,5 years,2023-11-02 02:49:53.843549,1.0,k foote,0,https://www.google.com/maps/contrib/1101870184...,[],2098.75
388,50093783,ChZDSUhNMG9nS0VJQ0FnSUNRcGEyMFFBEAE,The best Latin food in the area.,5 years,2023-11-02 02:49:53.844049,5.0,miguel gil,0,https://www.google.com/maps/contrib/1072434923...,[],2121.5625
389,50093783,ChZDSUhNMG9nS0VJQ0FnSUN3a1pxNUVBEAE,The Food Was Amazing. Everything had a fresh t...,5 years,2023-11-02 02:49:53.844547,5.0,Nick Paradise,4,https://www.google.com/maps/contrib/1164768180...,[],2144.375


In [77]:
dataframes = []
for restaurant_id in scrape_reviews.id.unique():  # For each restaurant id
    restaurant = scrape_reviews[scrape_reviews.id == restaurant_id].copy()

    place = scrape_reviews[scrape_reviews.id == restaurant_id]
    review_counts = pd.DataFrame(place.relative_date.value_counts())
    review_counts['increment'] = 365/review_counts.relative_date

    new_dates = []
    i = 0
    previous_year = '1 years'
    
    for date in restaurant.relative_date:  # For every date
        if 'years' in date:
            if date != previous_year:
                i = 0
                previous_year = date
            adjusted_date = (365*(int(date[0]))) + (review_counts.loc[date].increment * i)
            i += 1
            new_dates.append(round(adjusted_date))
        else:
            new_dates.append(date)
    restaurant['new_date'] = new_dates
    dataframes.append(restaurant)
    

In [83]:
dataframes

[           id                             id_review  \
 0    41375676   ChZDSUhNMG9nS0VJQ0FnSUNGMDhHSkVnEAE   
 1    41375676  ChdDSUhNMG9nS0VJQ0FnSUNGM2NhcGpnRRAB   
 2    41375676  ChdDSUhNMG9nS0VJQ0FnSUNGbmVidnRRRRAB   
 3    41375676   ChZDSUhNMG9nS0VJQ0FnSUNGNmY2Q0F3EAE   
 4    41375676   ChZDSUhNMG9nS0VJQ0FnSUNGNmZEdEVnEAE   
 ..        ...                                   ...   
 195  41375676  ChdDSUhNMG9nS0VJQ0FnSUNwM05mUjRnRRAB   
 196  41375676  ChdDSUhNMG9nS0VJQ0FnSUNwM0o3WDl3RRAB   
 197  41375676  ChdDSUhNMG9nS0VJQ0FnSUNwbkszQ3FnRRAB   
 198  41375676   ChZDSUhNMG9nS0VJQ0FnSUNwekxDNkJ3EAE   
 199  41375676   ChZDSUhNMG9nS0VJQ0FnSUNwOUxpQ1NnEAE   
 
                                                caption relative_date  \
 0                                                  NaN      11 hours   
 1                                                  NaN         a day   
 2                                                  NaN         a day   
 3                                